In [ ]:
#modüller
import pandas as pd
import config
import numpy as np
from datetime import datetime, date, timedelta
import requests
import json
from IPython.display import HTML
import os
import webbrowser
from jinja2 import Environment, FileSystemLoader
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta

In [ ]:
#fonksiyonlar

def fon_balances(t):

    fon_balances = pd.read_csv(config.FON_BALANCES_PATH, sep=";", decimal=",")

    fon_balances.index = pd.to_datetime(fon_balances["Tarih"], format="%d.%m.%Y")
    fon_balances.drop("Tarih", axis=1, inplace=True)
    fon_balances = fon_balances.resample('D').ffill()

    last_date = fon_balances.index.max().date()

    t = pd.to_datetime(t, dayfirst=True)
    new_dates = pd.date_range(last_date + timedelta(days=1), t)

    # Reindex the DataFrame with the new dates, forward filling values
    fon_balances = fon_balances.reindex(fon_balances.index.union(new_dates)).ffill()
    fon_balances.index.name = "Tarih"

    return fon_balances

def tcmb_fetcher(start_date, end_date, tickers=["TP.DK.USD.A.YTL-0"]):
    """ Fetches data from the TCMB (Central Bank of Turkey) website using POST requests with predefined headers. It retrieves data for specific tickers within a given date range, processes the JSON response, and returns a DataFrame with the fetched data.

    Parameters:
    - start_date: str, start date in the format "dd-mm-yyyy"
    - end_date: str, end date in the format "dd-mm-yyyy"
    - tickers: list of str, tickers to fetch data for

    Returns:
    - pd.DataFrame: DataFrame containing the fetched data with dates as index and tickers as columns
    """
    print("TCMB datası alınıyor...")
    # Replace dots with underscores in the start_date and end_date if necessary
    if "." in start_date:
        start_date = start_date.replace(".", "-")
    if "." in end_date:
        end_date = end_date.replace(".", "-")

    url = "https://evds2.tcmb.gov.tr/EVDSServlet"

    headers = {
        'Accept': '*/*',
        'Accept-Language': 'tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7',
        'Connection': 'keep-alive',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'Origin': 'https://evds2.tcmb.gov.tr',
        'Referer': 'https://evds2.tcmb.gov.tr/index.php?/evds/serieMarket',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest'
    }

    sd = pd.to_datetime(start_date, format="%d-%m-%Y")
    ed = pd.to_datetime(end_date, format="%d-%m-%Y")

    days = (ed - sd).days + 1  # include the end date in the range
    intervals = [(i, min(i + 20, days)) for i in range(0, days, 20)]

    # Prepare ticker columns for the DataFrame
    ticker_columns = [ticker.replace('.', '_')[:-2] for ticker in tickers]
    tcmb_df = pd.DataFrame(index=pd.date_range(start=sd, end=ed, freq="D"), columns=ticker_columns)

    for ticker in tickers:
        for start, end in intervals:
            data = {
                'orderby': 'Tarih+desc',
                'thousand': '1',
                'decimal': '2',
                'frequency': 'Date',
                'aggregationType': 'avg',
                'formula': '0',
                'graphicType': '0',
                'skip': start,
                'take': end - start,
                'sort': 'Tarih#true',
                'select': ticker,
                'startDate': start_date,
                'endDate': end_date,
                'obsCountEnabled': '',
                'obsCount': '',
                'categories': '5863',
                'mongoAdresses': 'evds',
                'userId': '',
                'datagroupString': 'bie_dkdovytl',
                'dateFormatValue': 'dd-mm-yyyy',
                'customFormula': 'null',
                'excludedSeries': 'null'
            }

            response = requests.post(url, headers=headers, data=data)
            response.raise_for_status()
            json_data = response.json()

            if 'items' not in json_data:
                print(f"Warning: No data found for interval {start} to {end}")
                continue

            d = pd.DataFrame(json_data["items"])
            if d.empty:
                print(f"Warning: DataFrame is empty for interval {start} to {end}")
                continue

            d.index = pd.to_datetime(d["Tarih"], format="%d-%m-%Y")
            d = d.drop(columns=["Tarih", "UNIXTIME"])
            tcmb_df.update(d)

    tcmb_df = tcmb_df.astype(float)
    tcmb_df.index.name = "TARIH"
    tcmb_df = tcmb_df.interpolate(method='linear', axis=0)
    print("TCMB datası başarıyla alındı.")
    return tcmb_df

def viop_fetcher(ticker, start_date, end_date):
    base_url = "https://www.borsaistanbul.com/data/vadeli/viop_gecici_tmp_{}.csv"

    start_date = datetime.strptime(start_date, "%d.%m.%Y")
    end_date = datetime.strptime(end_date, "%d.%m.%Y")
    start_date = start_date.strftime("%Y-%m-%d")
    end_date = end_date.strftime("%Y-%m-%d")
    print(f"VİOP datası alınıyor...")
    # Generate the date range
    date_range = pd.date_range(start=start_date, end=end_date, freq='B')

    # Initialize an empty list to store results
    results = []

    for d in date_range:

        formatted_date = d.strftime('%Y%m%d')
        #print(f'{formatted_date} için veri çekiliyor.')
        url = base_url.format(formatted_date)

        try:
            # Read the CSV data directly from the URL
            df = pd.read_csv(url, sep=';', encoding='latin1')

            # Filter the DataFrame for the given ticker
            filtered_df = df[df['SOZLESME KODU'] == ticker]

            # Check if there's a match and add the date and settlement price to the results
            if not filtered_df.empty:
                settlement_price = filtered_df['UZLASMA FIYATI'].values[0]
                results.append({'TARIH': d, ticker: settlement_price})

        except Exception as e:
            print(f"Error fetching data for date {formatted_date}: {e}")

    # Convert the results to a DataFrame
    result_df = pd.DataFrame(results)

    # Set the 'TARIH' column as datetime and make it the index
    result_df['TARIH'] = pd.to_datetime(result_df['TARIH'])
    result_df.set_index('TARIH', inplace=True)
    result_df = result_df.resample("D").asfreq()
    result_df[ticker] = pd.to_numeric(result_df[ticker], errors='coerce')
    result_df[ticker] = result_df[ticker].interpolate(method="linear")
    print("VİOP datası başarıyla alındı.")
    return result_df

def tefas_fetcher(tarih):
    kaydedilenler = []
    baslangic_tarihi, bitis_tarihi = pd.to_datetime(tarih, format="%d.%m.%Y")

    periyotlar = pd.date_range(baslangic_tarihi, bitis_tarihi, freq="90D")
    kalan_gun_sayisi = (bitis_tarihi - periyotlar[-1]).days

    print(f"{len(periyotlar)} periyot ve {kalan_gun_sayisi} kalan gün bulundu.")

    # Başlangıç ve bitiş tarihlerinin listelerini oluşturun
    baslangic_tarihleri = periyotlar[:-1].strftime("%d.%m.%Y").tolist()
    bitis_tarihleri = (periyotlar[1:] - timedelta(days=1)).strftime("%d.%m.%Y").tolist()
    # Kalan günleri ekleyin
    if kalan_gun_sayisi > 0:
        baslangic_tarihleri.append(periyotlar[-1].strftime("%d.%m.%Y"))
        bitis_tarihleri.append(bitis_tarihi.strftime("%d.%m.%Y"))
    else:
        baslangic_tarihleri = [tarih[0]]
        bitis_tarihleri = [tarih[1]]

    for baslangic, bitis in zip(baslangic_tarihleri, bitis_tarihleri):
        print(f"{baslangic} - {bitis} tarihleri arasındaki veriler çekiliyor...")

        url = config.TEFAS_URL
        essential_headers = {
            "Accept": "application/json, text/javascript, */*; q=0.01",
            "Accept-Language": "en-US,en;q=0.9,tr-TR;q=0.8,tr;q=0.7",
            "Connection": "keep-alive",
            "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
            "Referer": "https://www.tefas.gov.tr/TarihselVeriler.aspx",
        }

        data = {
            "fontip": "YAT",
            "sfontur": "",
            "fonkod": "",  # tüm kodları çeker
            "fongrup": "",
            "bastarih": baslangic,
            "bittarih": bitis,
            "fonturkod": "",
            "fonunvantip": ""
        }

        response = requests.post(url, headers=essential_headers, data=data)

        if response.status_code == 200:
            print("JSON dosyası uzak bilgisayardan başarıyla alındı...")
            fon_data = response.json()
            dosya_adi = f"{config.JSON_PATH}/{baslangic}-{bitis}_response_data.json"
            with open(dosya_adi, "w") as outfile:
                json.dump(fon_data, outfile, indent=4)
            print(f"{dosya_adi} kaydedildi.")
            kaydedilenler.append(dosya_adi)
        else:
            print(f"{baslangic}-{bitis} için JSON alma başarısız. Hata kodu: {response.status_code}")

    return kaydedilenler

def merge_and_interpolate(df1, df2):
    # df1 ve df2'yi birleştir, df2'nin değerleri df1'e öncelik verir
    df_combined = df2.combine_first(df1)

    # df_combined'a df1 ve df2'yi yeniden ekleyerek eksik tarihleri de tamamla
    df_final = pd.concat([df_combined, df1, df2]).groupby(level=0).first()
    df_final_interpolated = df_final.replace(0, np.nan).interpolate(method='linear')

    return df_final_interpolated

def update_price_db(guncel_datalar):
    """tefas dataları indirilen jsonlarının yer aldığı dosyaların path'lerini alır ve price_db'yi günceller.
    tcmb ve viop dataları doğrudan dataframe olarak alınır ve price_db'ye eklenir.
    [kaydedilenler, tcmb_df, viop_df] şeklinde
    """

    try:
        prices = pd.read_csv(config.PRICE_DB_PATH, sep=";", decimal=",", parse_dates=['TARIH'], dayfirst=True)
        prices.set_index('TARIH', inplace=True)
        prices.index = pd.to_datetime(prices.index)
    except FileNotFoundError:
        print(f"Hata: {config.PRICE_DB_PATH} dosyası bulunamadı.")
        return

    if guncel_datalar[0] is not False:
        for dosya in guncel_datalar[0]:
            try:
                print(f"İşleniyor: {dosya}")
                with open(dosya, "r") as f:
                    data = json.load(f)["data"]
                    df = pd.DataFrame(data)

                df = df.drop(columns=[
                    "BORSABULTENFIYAT", "FONUNVAN"
                ])

                df["TARIH"] = pd.to_datetime(df["TARIH"].astype(np.int64), unit="ms")

                df = df.sort_values(by=["FONKODU","TARIH"])

                df = df.pivot(index="TARIH", columns="FONKODU", values="FIYAT")

            except (ValueError, TypeError) as e:
                print(f"Hata: '{dosya}' dosyası işlenirken hata oluştu: {e}")
                continue

            prices = merge_and_interpolate(prices, df)

    if guncel_datalar[1] is not False:
        prices = merge_and_interpolate(prices, guncel_datalar[1])

    if  guncel_datalar[2] is not False:
        prices = merge_and_interpolate(prices, guncel_datalar[2])

    # buraya tefas fonları dışındaki varlıkların satılması sırasındaki fiyatları manuel olarak sell_value_price_db fonksiyonu ile güncellenmeli.
    prices.to_csv(config.PRICE_DB_PATH, sep=";", decimal=",")
    print("price_db.csv dosyası kaydedildi.")
    return prices

def hist_price_update(dir=config.JSON_PATH):
    print(f"İşleniyor: {dir}")
    try:
        prices = pd.read_csv(config.PRICE_DB_PATH, sep=";", decimal=",", parse_dates=['TARIH'], dayfirst=True)
        prices.set_index('TARIH', inplace=True)
        prices.index = pd.to_datetime(prices.index)
    except FileNotFoundError:
        print(f"Hata: {config.PRICE_DB_PATH} dosyası bulunamadı.")
        return

    #loop through json file in the directory
    for file in os.listdir(dir):

        if file.endswith(".json"):
            print(f"İşleniyor: {file}")
            with open(f"{dir}/{file}", "r") as f:
                data = json.load(f)["data"]
                df = pd.DataFrame(data)
                df["TARIH"] = pd.to_datetime(df["TARIH"].astype(np.int64), unit="ms")
                df = df.sort_values(by=["FONKODU","TARIH"])
                df = df.pivot(index="TARIH", columns="FONKODU", values="FIYAT")
            prices = merge_and_interpolate(prices, df)
    prices.to_csv(config.PRICE_DB_PATH, sep=";", decimal=",")

def generate_portfolio_db(fon_balances, currency='TRY', prices=None):
    """
    Generates a portfolio database with various calculated fields.

    Args:
        fon_balances (pd.DataFrame): DataFrame with fund balances.
        currency (str, optional): The currency for calculations ('TRY' or 'USD').
                                    Defaults to 'TRY'.
        prices (pd.DataFrame, optional): DataFrame with price data.
                                        If None, loads from config.PRICE_DB_PATH.

    Returns:
        pd.DataFrame: The generated portfolio database.
    """

    if prices is None:
        prices = pd.read_csv(config.PRICE_DB_PATH, sep=";", decimal=",", parse_dates=['TARIH'], dayfirst=True)
        prices.set_index('TARIH', inplace=True)
        prices.index = pd.to_datetime(prices.index)

    prices = prices.fillna(0)

    if currency == 'USD':
        prices = prices.div(prices["TP_DK_USD_A_YTL"], axis=0)

    prices = prices[fon_balances.columns]
    prices = prices.loc[fon_balances.index]

    prices = prices.add_suffix("_p").fillna(0)
    fon_balances = fon_balances.add_suffix("_q").fillna(0)

    df = pd.concat([fon_balances, prices], axis=1)
    df.fillna(0, inplace=True)

    # Calculate total value for each asset
    for col in df.columns:
        if col.endswith('_q'):
            base_name = col[:-2]
            p_col = base_name + '_p'
            v_col = base_name + '_v'
            if p_col in df.columns:
                df[v_col] = df[col] * df[p_col]

    # Calculate portfolio value
    df[f'Portfolio_{currency}'] = df.filter(like='_v').sum(axis=1)

    # Calculate price differences and profits
    for col in df.columns:
        if col.endswith('_p'):
            df[col[:-2] + '_diff'] = df[col].diff()
    for col in df.columns:
        if col.endswith('_diff'):
            df[col[:-5] + '_profit'] = df[col] * df[col[:-5] + '_q'].shift(1)

    # Calculate portfolio differences and profits
    df[f"Portfolio_{currency}_diff"] = df[f"Portfolio_{currency}"].diff()
    df[f'Portfolio_Profit_{currency}'] = df.filter(like='_profit').sum(axis=1)
    df[f'Portfolio_Profit_{currency}_daily'] = df[f'Portfolio_Profit_{currency}'] / df[f'Portfolio_{currency}'].shift(1)
    df[f'Portfolio_Profit_{currency}_cum'] = df[f'Portfolio_Profit_{currency}'].cumsum()

    # Calculate cash flow
    df[f"CF_{currency}"] = df[f"Portfolio_{currency}_diff"] - df[f"Portfolio_Profit_{currency}"]
    df[f"CF_{currency}"].iloc[0] = df[f"Portfolio_{currency}"].iloc[0]

    return df

def add_invested_capital(df, window_size, currency='TRY'):
    """
    Adds a column for average invested capital over a given window.

    Args:
        df (pd.DataFrame): The input DataFrame.
        window_size (int): The size of the rolling window.
        currency (str, optional): The currency code to use in column names.
                                 Defaults to "TRY".

    Returns:
        pd.DataFrame: The DataFrame with the added column.
    """

    invested_capital = []
    for i in range(len(df)):
        if i + 1 < window_size:
            invested_capital.append(pd.NA)
        else:
            portfolio_window = df[f'Portfolio_{currency}'].iloc[i+1-window_size:i+1]
            cf_window = df[f'CF_{currency}'].iloc[i+1-window_size:i+1]
            combined_window = pd.concat([pd.Series([portfolio_window.iloc[0]]), cf_window.iloc[1:]])
            invested_capital.append(combined_window.cumsum().mean())

    df[f'Invested_Capital_{currency}_{window_size}_avg'] = invested_capital
    return df

def add_roic_column(df, window_size, currency='TRY'):
    """
    Adds a column for ROIC over a given window.

    Args:
        df (pd.DataFrame): The input DataFrame.
        window_size (int): The size of the rolling window.
        currency (str, optional): The currency code to use in column names.
                                 Defaults to "TRY".

    Returns:
        pd.DataFrame: The DataFrame with the added column.
    """

    profit_cumsum = []
    for i in range(len(df)):
        if i + 1 < window_size:
            profit_cumsum.append(pd.NA)
        else:
            profit_window = df[f'Portfolio_Profit_{currency}'].iloc[i+1-window_size:i+1]
            profit_cumsum.append(profit_window.cumsum().iloc[-1])

    column_name = f'Portfolio_Profit_{currency}_sum_{window_size}'
    df[column_name] = profit_cumsum
    df[f'Roic_{window_size}'] = df[column_name] / df[f'Invested_Capital_{currency}_{window_size}_avg']
    return df

def roics(df, d, currency='TRY'):
    """
    Calculates various ROIC metrics for a given DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame containing financial data.
        d (str): A date string in the format YYYY-MM-DD.
        currency (str, optional): The currency code to use as a prefix in column names.
                             Defaults to "TRY".

    Returns:
        pd.DataFrame: The DataFrame with calculated ROIC metrics.
    """

    df[f"Invested_Capital_{currency}_cum"] = df[f"CF_{currency}"].cumsum()
    df[f"Invested_Capital_{currency}_avg"] = df[f"Invested_Capital_{currency}_cum"].expanding().mean()
    df["ROIC"] = df[f"Portfolio_Profit_{currency}_cum"] / df[f"Invested_Capital_{currency}_avg"]

    df['days'] = range(len(df))
    df["ROIC_CAGR"] = (1 + df["ROIC"]) ** (365 / df["days"]) - 1

    df = add_invested_capital(df, 30, currency)
    df = add_roic_column(df, 30, currency)

    df = add_invested_capital(df, 365, currency)
    df = add_roic_column(df, 365, currency)

    days_passed = (pd.to_datetime(d) - pd.to_datetime(d).replace(month=1, day=1)).days
    df = add_invested_capital(df, days_passed, currency)
    df = add_roic_column(df, days_passed, currency)

    return df

def add_sharpe_column(df, window_size, currency='TRY'):
    # calculate daily pct changes from PPZ_p column
    df["PPZ_pct_change"] = df["PPZ_p"].pct_change()
     # calculate std of Portfolio_Profit_{currency}_daily
    df[f"Portfolio_Profit_{currency}_std_{window_size}"] = df[f"Portfolio_Profit_{currency}_daily"].rolling(window=window_size).std()
    # calculate excess return
    df[f"Excess_Return_{currency}_{window_size}"] = df[f"Portfolio_Profit_{currency}_daily"] - df["PPZ_pct_change"]
    # calculate rolling avg and std of excess return
    df[f"Excess_Return_{currency}_{window_size}_avg"] = df[f"Excess_Return_{currency}_{window_size}"].rolling(window=window_size).mean()
    df[f"Excess_Return_{currency}_{window_size}_std"] = df[f"Excess_Return_{currency}_{window_size}"].rolling(window=window_size).std()
    # calculate Sharpe Ratio and multiply with sqrt(365) to annualize
    df[f"Sharpe_Ratio_{currency}_{window_size}"] = (df[f"Excess_Return_{currency}_{window_size}_avg"] / df[f"Excess_Return_{currency}_{window_size}_std"]) * np.sqrt(365)

    #df.to_csv("df.csv", sep=";", decimal=",")
    return df

def format_percentages(df):
    def format_value(x):
        """Formats a value as a percentage, handling NaN and infinite values."""
        if pd.isna(x):  # Check for both pd.NA and np.nan
            return "n/a"
        elif not np.isfinite(x):  # Use np.isfinite to check for infinite values
            return "Inf"
        elif abs(x) < 0.01:
            return f"{x * 100:.3f}%"
        elif 0.01 <= abs(x) < 1:
            return f"{x * 100:.2f}%"
        else:
            return f"{x * 100:.0f}%"

    return df.applymap(format_value)

def generate_dates(d,t=10):
    #convert d string to datetime and substract 10 days
    d = pd.to_datetime(d, dayfirst=True) - timedelta(days=t-1)
    dates = pd.date_range(d, periods=t, freq='D')
    dates = dates.strftime("%d.%m.%Y")
    #convert to list
    dates = dates.tolist()
    return dates

def getiri_tab(date, currencies=['TRY']):
    """
    Generates a list of HTML tables showing portfolio returns for different periods.

    Args:
        date (str): The date for which to calculate returns (DD.MM.YYYY).
        currencies (list, optional): List of currencies to calculate for.
                                     Defaults to ['TRY'].

    Returns:
        list: A list of tuples, each containing a title and an HTML table string.
    """

    fon_balances_data = fon_balances(date)  # Assuming fon_balances is defined elsewhere
    dataframes_with_titles = []

    for currency in currencies:
        portfolio_db = generate_portfolio_db(fon_balances_data, currency=currency)
        roics_data = roics(portfolio_db, date, currency=currency)
        roics_data_plot = roics_data
        roics_data = format_percentages(roics_data)

        # Select columns containing "ROIC" (case-insensitive)
        roicz = roics_data.columns[roics_data.columns.str.contains("ROIC", case=False)].tolist()

        # Manuel olarak bugünün tarihini girin (örneğin: "31.05.2024")
        today_str = "31.05.2024"
        today_date = datetime.strptime(today_str, '%d.%m.%Y')

        months_dates = [
            (today_str if i == 0 else (today_date.replace(day=1) - relativedelta(months=i) - timedelta(days=1)).strftime('%d.%m.%Y'))
            for i in range(12)
        ]
        #reverse the list
        months_dates = months_dates[::-1]

        roic_tab_months = roics_data.loc[months_dates, roicz]

        # Daily data
        days_dates = generate_dates(date)  # Assuming generate_dates is defined elsewhere
        roicz_days = [f"Portfolio_Profit_{currency}_daily"] + roicz
        roic_tab_days = roics_data.loc[days_dates, roicz_days]

        # Yearly data
        years_dates = ["31.12.2020", "31.12.2021", "31.12.2022", "31.12.2023", "31.05.2024"]
        roic_tab_years = roics_data.loc[years_dates, roicz].drop("Roic_30", axis=1, errors='ignore')

        # Drop rows with NA/NaN values in 'Roic_365'
        roics_data_plot_cleaned = roics_data_plot.dropna(subset=["Roic_365"])

        # Convert to float in case of any type issues
        roics_data_plot_cleaned["Roic_365"] = roics_data_plot_cleaned["Roic_365"].astype(float)

        # plot style
        plt.style.use('seaborn-darkgrid')
        # Plotting the cleaned data
        plt.figure(figsize=(10, 6))
        plt.plot(roics_data_plot_cleaned.index, roics_data_plot_cleaned["Roic_365"], label="ROIC")
        plt.title(f"ROIC_365 - {currency}")
        plt.xlabel("Date")
        plt.ylabel("ROIC")
        plt.grid(True)
        plt.legend()
        plt.savefig(f"roic_{currency}.png")
        plt.close()

        # generate html code for the plot
        plot_html = f"<img src='roic_{currency}.png'>"

        # Add dataframes to the list
        dataframes_with_titles.extend([
            (f"Son 10 Gün - {currency}", roic_tab_days.to_html()),
            (f"Ay Sonları - {currency}", roic_tab_months.to_html()),
            (f"Yıl Sonları - {currency}", roic_tab_years.to_html()),
            (f"Tarihsel ROIC - {currency}", plot_html)
        ])

    return dataframes_with_titles

def risk_tab(date, currencies=['TRY']):
    """
    Generates a list of HTML tables showing portfolio Sharpe ratios for different periods.

    Args:
        date (str): The date for which to calculate returns (DD.MM.YYYY).
        currencies (list, optional): List of currencies to calculate for.
                                     Defaults to ['TRY'].

    Returns:
        list: A list of tuples, each containing a title and an HTML table string.
    """

    fon_balances_data = fon_balances(date)  # Assuming fon_balances is defined elsewhere
    dataframes_with_titles = []

    for currency in currencies:
        portfolio_db = generate_portfolio_db(fon_balances_data, currency=currency)

        # Adding Sharpe Ratio columns to the portfolio_db
        portfolio_db = add_sharpe_column(portfolio_db, 30, currency)
        portfolio_db = add_sharpe_column(portfolio_db, 365, currency)

        # Formatting the Sharpe Ratio columns as one decimal float
        portfolio_db[f"Sharpe_Ratio_{currency}_30"] = portfolio_db[f"Sharpe_Ratio_{currency}_30"].round(1)
        portfolio_db[f"Sharpe_Ratio_{currency}_365"] = portfolio_db[f"Sharpe_Ratio_{currency}_365"].round(1)

        # Selecting columns containing "Sharpe" (case-insensitive)
        sharpe_cols = portfolio_db.columns[portfolio_db.columns.str.contains("Sharpe", case=False)].tolist()

        # Daily data
        days_dates = generate_dates(date)
        sharpe_tab_days = portfolio_db.loc[days_dates, sharpe_cols]

        # Yearly data
        years_dates = ["31.12.2020", "31.12.2021", "31.12.2022", "31.12.2023", "31.05.2024"]
        sharpe_tab_years = portfolio_db.loc[years_dates, sharpe_cols]

        # Monthly data
        today_str = "31.05.2024"
        today_date = datetime.strptime(today_str, '%d.%m.%Y')

        months_dates = [
            (today_str if i == 0 else (today_date.replace(day=1) - relativedelta(months=i) - timedelta(days=1)).strftime('%d.%m.%Y'))
            for i in range(12)
        ]
        months_dates = months_dates[::-1]
        sharpe_tab_months = portfolio_db.loc[months_dates, sharpe_cols]

        dataframes_with_titles.extend([
            (f"Son 10 Gün - {currency}", sharpe_tab_days.to_html()),
            (f"Ay Sonları - {currency}", sharpe_tab_months.to_html()),
            (f"Yıl Sonları - {currency}", sharpe_tab_years.to_html())
        ])

    return dataframes_with_titles


#fonksiyonlar son

In [ ]:
#datafetch
#her türlü tarih girdisi formatı dd.mm.yyyy olmalıdır.
tarih = ["29.05.2024", "31.05.2024"]
tefas = tefas_fetcher(tarih)
tcmb = tcmb_fetcher(tarih[0], tarih[1], tickers=["TP.DK.USD.A.YTL-0"])
viop = viop_fetcher("F_XU0300624", tarih[0], tarih[1])
prices = update_price_db([tefas, tcmb, viop])

In [ ]:
d = "31.05.2024"
dataframes_with_titles_getiri = getiri_tab(d, currencies=['TRY', 'USD'])

dataframes_with_titles_risk = risk_tab(d, currencies=['TRY'])

# Zaman damgasını al
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M')

# Jinja2 ortamını oluştur ve şablonu yükle
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('template.html')

# Şablonu doldur ve HTML dosyasını oluştur
html_content = template.render(getiri_tables=dataframes_with_titles_getiri,risk_tables=dataframes_with_titles_risk, timestamp=timestamp)

# HTML içeriğini bir dosyaya yaz
with open('Portföy Performans Raporu.html', 'w', encoding="utf-8") as file:
    file.write(html_content)

# HTML dosyasını aç
webbrowser.open('Portföy Performans Raporu.html')


In [ ]:
#fonksiyonları çalıştır
fon_balances_data = fon_balances("31.05.2024")

portfolio_db = generate_portfolio_db(fon_balances_data, currency="TRY")
d = add_sharpe_column(portfolio_db, 30, currency="TRY")
#print last 5 columns
print(d.iloc[:,-2:])
